In [1]:
import numpy as np
import copy

In [2]:
# Note - acceleration and deceleration will be equal
BUS_LENGTH = 18.3 # https://en.wikipedia.org/wiki/Classic_(transit_bus)
BUS_ACCELERATION = 2 # https://hypertextbook.com/facts/2001/MeredithBarricella.shtml
CAR_LENGTH = 4.7 # https://en.wikipedia.org/wiki/Mid-size_car
CAR_ACCELERATION = 4 # https://hypertextbook.com/facts/2001/MeredithBarricella.shtml

## Define the roadway


# LIGHT_LOCATIONS = [500, 1000]
# LIGHT_DUTIES = [1, 1]
# LIGHT_CYCLES = [60, 60]
# LIGHT_PHASES = [0, 0]


LIGHT_LOCATIONS = [100, 500, 1200, 1400, 2000] # Vehicles enter at zero meters
LIGHT_DUTIES = [0.6, 0.8, 0.5, 0.6, 0.7]
LIGHT_CYCLES = [60, 40, 60, 40, 60]
LIGHT_PHASES = [0, 0, 0, 0, 0]


## Simulation parameters
TIMESTEP = 0.1
NUM_LANES = 1
SPEED_LIMIT = 30 # Defines the max speed for all vehicles
SPACING = 5 # meters between cars
STARTING_SPEED = SPEED_LIMIT/2
SIMULATION_DURATION = 7200 # two hours
CARS_PER_SECOND = 1/60
PERCENT_OF_BUSSES = 0.005

In [3]:
 # globals to store exited vehicle information
DURATIONS = []
STOP_DURATIONS = []

In [4]:
class Light(object):
    '''
    Represents a traffic signal
    
    var: loc, where it is on the street
    var: duty, percent of cycle the signal is green
    var: cycle, duration of cycle in seconds
    var: phase, offset from time zero of cycle start, in seconds
    '''
    def __init__(self, loc, duty, cycle, phase):
        self.loc = loc
        self.duty = duty
        self.cycle = cycle
        self.phase = phase
        return
    
    def green(self, time):
        '''
        Returns true if the light is green at a given time
        '''
        if ((time-self.phase) % self.cycle) < (self.duty*self.cycle):
            return True
        else:
            return False

In [5]:
class Vehicle(object):
    '''
    Represents a vehicle on the road
    
    var: max_speed, the fastest a vehicle will go (m/s)
    var: acceleration, the acceleration (m/s^2)
    var: length, the length of the vehicle
    
    var: speed, the current speed of the vehicle
    var: position, the current position of the front of the vehicle
    var: stopping_distance, the current stopping distance of the vehicle
    
    var: lane, the lane the vehicle occupies, index not object
    
    var: duration, the time since the vehicle entered the roadway
    var: time_stopped, the time the vehicle spent stopped
    '''
    def __init__(self, is_bus, max_speed, acceleration, length, lane_idx, start=0, speed=STARTING_SPEED, position=0):
        '''
        Fixed and starting values
        '''
        self.is_bus = is_bus
        self.max_speed = max_speed
        self.acceleration = acceleration
        self.length = length
        
        self.lane_idx = lane_idx
        
        self.speed = speed
        self.position = position
        self.update_stopping_distance()
        
        self.start = start
        
        self.duration = 0
        self.time_stopped = 0
        
        return
        
    def step(self, road, time_step):
        '''
        param: lights, list of Light objects on the roadway
        param: road, Road object that holds lanes of the road
        param: time_step, duration of time to simulate
        '''
        my_lane = road.lanes[self.lane_idx]
        # vf^2 = vi^2 + 2*a*d, a is negative
        self.update_stopping_distance()
        
        obstructed = my_lane.obstructed(self.position, self.stopping_distance)
        stopped = road.stopped(self.position, self.stopping_distance)
        
        ## TODO: incorporate lane changing
        if not (obstructed or stopped):
            self.accelerate(time_step)
        else:
            self.decelerate(time_step)
        return
    
    def update_stopping_distance(self):
        self.stopping_distance = (self.speed**2)/(2*self.acceleration)
        return
    
    def accelerate(self, time_step):
        if self.speed < self.max_speed:
            self.speed += time_step*self.acceleration
            if self.speed > self.max_speed:
                self.speed = self.max_speed
        self.move(time_step)
        return
    
    def decelerate(self, time_step):
        if self.speed > 0:
            self.speed -= time_step*self.acceleration
            if self.speed < 0:
                self.speed = 0
        else:
            self.time_stopped += time_step
        self.move(time_step)
        return
    
    def move(self, time_step):
        self.position += self.speed*time_step
        self.duration += time_step
        # Troubleshooting
#         print("{}: {}".format("bus" if self.is_bus else "car", self.position))
        return
    
    def exit(self):
        DURATIONS.append(self.duration)
        STOP_DURATIONS.append(self.time_stopped)
        return

In [6]:
class Bus(Vehicle):
    """
    Represents a bus
    """
    def __init__(self, lane, start, speed=STARTING_SPEED):
        Vehicle.__init__(self, True, SPEED_LIMIT, BUS_ACCELERATION, BUS_LENGTH, lane, start, speed)
        return
    
class Car(Vehicle):
    """
    Represents a car
    """
    def __init__(self, lane, start, speed=STARTING_SPEED):
        Vehicle.__init__(self, False, SPEED_LIMIT, CAR_ACCELERATION, CAR_LENGTH, lane, start, speed)
        return

In [7]:
class Lane(object):
    '''
    Represents a lane that holds vehicles
    
    var: end, position of end of lane
    var: vehicles, list of car objects in the lane, sorted in reverse order of position
    var: q, vehicles that haven't entered yet
    '''
    def __init__(self, end):
        self.end = end
        self.vehicles = []
        self.q = []
        return
    
    def add_vehicle(self, vehicle):
        self.vehicles.append(vehicle)
        return
    
    def remove_vehicle(self):
        self.vehicles.pop(0)
        return
    
    def check_for_exits(self):
        if self.vehicles:
            while(self.vehicles[0].position > self.end):
                self.vehicles[0].exit()
                self.remove_vehicle()
                if not self.vehicles:
                    break
        return
    
    def lane_step(self, road, time_step):
        for vehicle in self.vehicles:
            vehicle.step(road, time_step)
        self.check_for_exits()
        if self.q:
            if not self.obstructed(self.q[0].position, self.q[0].stopping_distance):
                self.add_vehicle(self.q.pop(0))
        return
    
    def obstructed(self, position, stopping_distance):
        if not self.vehicles:
            return False
        vIdx = 0
        while(True):
            next_vehicle = self.vehicles[vIdx]
            if next_vehicle.position == position:
                return False # Unobstructed by vehicles in front
            elif next_vehicle.position + next_vehicle.stopping_distance - SPACING - next_vehicle.length \
              < stopping_distance + position: # Stop if front of your vehicle violate space for vehicle in front
                return True
            else:
                vIdx += 1
            if vIdx == len(self.vehicles): # vehicle in queue
                return False
        assert(False)

In [8]:
class Road(object):
    '''
    Represents a road with multiple lanes and lights
    
    var: lights, list of light objects
    var: lanes, list of lane objects
    var: time, the time in seconds of the simulation
    '''
    def __init__(self, light_locations, light_duties, light_cycles, light_phases, lanes, time=0):
        '''
        param: light_*, light parameters
        param: lanes, number of lanes
        '''
        self.lights = []
        for loc, duty, cycle, phase in zip(light_locations, light_duties, light_cycles, light_phases):
            self.lights.append(Light(loc,duty,cycle,phase))
        
        self.lanes = []
        end = light_locations[-1]
        for lane in range(lanes):
            self.lanes.append(Lane(end))
            
        self.time = 0
        return
    
    # Assumes cars will slow down for only the nearest light
    def stopped(self, position, stopping_distance):
        lIdx = 0
        while(True):
# Troubleshooting vehicles passing each other
#             if lIdx == len(self.lights):
#                 print(position, stopping_distance)
#                 for lane in self.lanes:
#                     for vehicle in lane.vehicles:
#                         print("{}, {}".format(vehicle.position, vehicle.is_bus))
            if self.lights[lIdx].loc <= position:
                lIdx += 1
                continue
            if self.lights[lIdx].green(self.time):
                return False
            elif self.lights[lIdx].loc > position + stopping_distance:
                return True # sufficient space to stop
            else:  
                return False # insufficient space to stop
        assert(False)
        
    def road_step(self, time_step):
        for lane in self.lanes:
            lane.lane_step(self, time_step)
        self.time += time_step
        return
    
    def simulate(self, new_vehicles, time_step):
        vehicles = copy.copy(new_vehicles)
        while(True):
            if vehicles:
                for vehicle in copy.copy(vehicles): # Create copy b/c removal within for loop
                    if vehicle.start <= self.time:        
                        vehicles.remove(vehicle)
                        self.lanes[vehicle.lane_idx].q.append(vehicle)
            else: # Once all vehicles have been added to road, check for empty road
                empty_road = True
                for lane in self.lanes:
                    if lane.vehicles:
                        empty_road = False
                if empty_road:
                    return
            self.road_step(time_step)

In [9]:
def add_car_or_bus(percent_of_busses, start):
    choice = np.random.rand()
    if choice < percent_of_busses:
        return Bus(np.random.choice(NUM_LANES), start)
    else:
        return Car(np.random.choice(NUM_LANES), start)

In [10]:
new_vehicles = []
# new_vehicles.append(Car(np.random.choice(NUM_LANES), start=1))
# new_vehicles.append(Bus(np.random.choice(NUM_LANES), start=0))

for second in range(SIMULATION_DURATION):
    if np.random.rand() < CARS_PER_SECOND:
        new_vehicles.append(add_car_or_bus(PERCENT_OF_BUSSES, second))


In [11]:
road = Road(LIGHT_LOCATIONS, LIGHT_DUTIES, LIGHT_CYCLES, LIGHT_PHASES, NUM_LANES)
road.simulate(new_vehicles, TIMESTEP)

In [12]:
DURATIONS

[100.99999999999854,
 102.29999999999846,
 96.89999999999877,
 130.49999999999687,
 126.1999999999971,
 115.5999999999977,
 130.99999999999685,
 137.0999999999965,
 132.29999999999677,
 115.29999999999772,
 104.89999999999831,
 122.6999999999973,
 140.99999999999628,
 109.89999999999803,
 141.99999999999622,
 141.79999999999623,
 141.99999999999622,
 115.5999999999977,
 146.99999999999594,
 141.79999999999623,
 110.399999999998,
 142.09999999999621,
 148.79999999999583,
 112.99999999999785,
 101.89999999999849,
 100.19999999999858,
 123.29999999999727,
 99.69999999999861,
 113.8999999999978,
 116.39999999999766,
 144.0999999999961,
 138.9999999999964,
 132.09999999999678,
 110.89999999999797,
 107.59999999999816,
 133.09999999999673,
 123.59999999999725,
 94.89999999999888,
 137.69999999999646,
 123.69999999999725,
 110.79999999999798,
 144.0999999999961,
 123.59999999999725,
 124.4999999999972,
 146.99999999999594,
 128.099999999997,
 145.999999999996,
 141.79999999999623,
 130.299999

In [13]:
STOP_DURATIONS

[22.40000000000005,
 22.40000000000005,
 11.199999999999976,
 37.80000000000027,
 31.200000000000173,
 32.800000000000196,
 41.100000000000314,
 41.30000000000032,
 36.50000000000025,
 32.800000000000196,
 14.499999999999964,
 33.2000000000002,
 45.300000000000374,
 27.100000000000115,
 46.30000000000039,
 47.0000000000004,
 46.700000000000394,
 32.800000000000196,
 51.30000000000046,
 47.0000000000004,
 20.50000000000002,
 47.90000000000041,
 52.80000000000048,
 32.800000000000196,
 11.499999999999975,
 11.499999999999975,
 33.2000000000002,
 22.40000000000005,
 31.100000000000172,
 26.500000000000107,
 49.90000000000044,
 43.300000000000345,
 37.90000000000027,
 28.10000000000013,
 17.399999999999977,
 38.90000000000028,
 32.800000000000196,
 11.199999999999976,
 40.400000000000304,
 33.80000000000021,
 20.900000000000027,
 49.90000000000044,
 32.800000000000196,
 33.60000000000021,
 51.30000000000046,
 33.90000000000021,
 50.300000000000445,
 47.0000000000004,
 41.200000000000315,
 